In [7]:
import numpy as np

covs_all = np.load('../data/simulation2/covs_all.npy')
cov = np.load('../data/simulation2/cov_pop.npy')

In [8]:
num_cells = 10
len_series = np.array([68, 116, 216, 216,  94, 216, 216, 190, 216, 216],dtype=np.int64)

In [9]:
ns01 = np.zeros((21,34),dtype=np.int64)

In [10]:
list0 = [0,3,4,6,7,9,10,11,12,13,15,18,19,20]
list1 = [2,7,8,9,12,14,15,16,17,18,20,24,25,26,28,31,32,33]
list2 = [2,4,5,6,7,8,9,10]

In [11]:
for i in list0:
    for j in list1:
        print(i,j)
        eigvals = np.zeros((num_cells,2))
        sqrt_eigvals = np.zeros((num_cells,2))
        eigvecs = np.zeros((num_cells,2,2))
        for c in range(num_cells):
            #mat = np.array([[S00[c],covs[c,0,1],covs[c,0,2],covs[c,0,6]],[covs[c,1,0],covs[c,1,1],covs[c,1,2],covs[c,1,6]],[covs[c,2,0],covs[c,2,1],covs[c,2,2],covs[c,2,6]],[covs[c,6,0],covs[c,6,1],covs[c,6,2],covs[c,6,6]]])
            #eigvals[c], eigvecs[c] = np.linalg.eigh(mat)
            eigvals[c], eigvecs[c] = np.linalg.eigh(covs_all[np.ix_([c],[i+4,j+25],[i+4,j+25])])
            sqrt_eigvals[c] = np.sqrt(eigvals[c])

        num_trials = 5000000
        vals = np.zeros((num_trials,num_cells,2))
        for tr in range(num_trials):
            #if tr%10000==0: print(tr)
            for c in range(num_cells):
                vals[tr,c] = eigvecs[c]@(sqrt_eigvals[c]*np.random.normal(size=(2,)))
        
        means_sim = np.zeros((num_trials,2))
        means_sim = np.sum(vals*len_series[np.newaxis,:,np.newaxis], axis=1)/np.sum(len_series[:])
        
        covs_sim = np.zeros((num_trials,2,2))
        for k in range(2):
            for l in range(2):
                covs_sim[:,k,l] = np.sum(len_series[np.newaxis,:]*(vals[:,:,k]-means_sim[:,np.newaxis,k])*(vals[:,:,l]-means_sim[:,np.newaxis,l]), axis=1)/np.sum(len_series[:])
        covs_diff = cov[np.ix_([i+4,j+25],[i+4,j+25])][np.newaxis,:,:] - covs_sim
        
        #scale_mat_inv = np.linalg.inv(scale_mat[np.ix_([i,j],[i,j])])
        #mat_sim = np.matmul(covs_diff, scale_mat_inv)
        a = covs_diff[:,0,0]
        b = covs_diff[:,0,1]
        c = covs_diff[:,1,0]
        d = covs_diff[:,1,1]
        eigs_sim = (a+d-np.sqrt((a-d)**2+4*b*c))/2
        ns01[i,j] = np.sum(np.where(eigs_sim<=0,1,0))
        print(i,j,ns01[i,j])

0 2
0 2 95988
0 7
0 7 65997
0 8
0 8 1891
0 9
0 9 30131
0 12
0 12 134473
0 14
0 14 7328
0 15
0 15 2308
0 16
0 16 1159
0 17
0 17 2361
0 18
0 18 21342
0 20
0 20 13876
0 24
0 24 1231
0 25
0 25 66091
0 26
0 26 56813
0 28
0 28 85134
0 31
0 31 4662
0 32
0 32 6066
0 33
0 33 8844
3 2
3 2 360894
3 7
3 7 2041052
3 8
3 8 8375
3 9
3 9 72353
3 12
3 12 103306
3 14
3 14 350995
3 15
3 15 3589
3 16
3 16 2037
3 17
3 17 1488
3 18
3 18 124478
3 20
3 20 24675
3 24
3 24 10941
3 25
3 25 11978
3 26
3 26 164129
3 28
3 28 13053
3 31
3 31 3249
3 32
3 32 7006
3 33
3 33 1997
4 2
4 2 279
4 7
4 7 46105
4 8
4 8 0
4 9
4 9 52116
4 12
4 12 160
4 14
4 14 16458
4 15
4 15 1
4 16
4 16 0
4 17
4 17 1
4 18
4 18 13362
4 20
4 20 12996
4 24
4 24 2
4 25
4 25 6799
4 26
4 26 982
4 28
4 28 614
4 31
4 31 65
4 32
4 32 1037
4 33
4 33 0
6 2
6 2 2
6 7
6 7 20709
6 8
6 8 486
6 9
6 9 17738
6 12
6 12 102
6 14
6 14 355340
6 15
6 15 0
6 16
6 16 34350
6 17
6 17 444917
6 18
6 18 4004097
6 20
6 20 1102042
6 24
6 24 14
6 25
6 25 15906
6 26
6 26 1730

In [12]:
with open('../data/simulation2/ns01.npy','wb') as f:
    np.save(f, ns01)